In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.formula.api import ols # Creating linear model
import scipy.stats as stats

pd.set_option('display.max_columns', 500)

In [3]:
credit = pd.read_csv('credit_altered.csv')
credit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_SEP,PAY_AUG,PAY_JUL,PAY_JUN,PAY_MAY,PAY_APR,BILL_AMT_SEP,BILL_AMT_AUG,BILL_AMT_JUL,BILL_AMT_JUN,BILL_AMT_MAY,BILL_AMT_APR,PAY_AMT_SEP,PAY_AMT_AUG,PAY_AMT_JUL,PAY_AMT_JUN,PAY_AMT_MAY,PAY_AMT_APR,DEFAULT,RISK_VALUE,BILL_VALUE
0,1,20000,Female,University,Married,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1,-2,-3102
1,2,120000,Female,University,Single,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1,3,-11395
2,3,90000,Female,University,Single,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,0,-66396
3,4,50000,Female,University,Married,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0,0,-176956
4,5,50000,Male,University,Married,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0,-2,-42352


In [4]:
credit['SEX_ED'] = credit['SEX']+"_"+credit['EDUCATION']
credit['SEX_MAR'] = credit['SEX']+"_"+credit['MARRIAGE']

In [12]:
group = pd.DataFrame(credit.groupby('SEX_ED')['DEFAULT'].value_counts()).rename(columns = {'DEFAULT':'COUNT'}).reset_index()
group

,SEX_ED,DEFAULT,COUNT
0,Female_Graduate,0,5101
1,Female_Graduate,1,1130
2,Female_High School,0,2235
3,Female_High School,1,692
4,Female_Others,0,78
5,Female_Others,1,3
6,Female_University,0,6734
7,Female_University,1,1922
8,Female_Unknown,0,201
9,Female_Unknown,1,16


- FEMALE and MALE both in education category others and unknown category are bad groups.

In [16]:
# WOE and IV
l = group['SEX_ED'].unique()
l

array(['Female_Graduate', 'Female_High School', 'Female_Others',
       'Female_University', 'Female_Unknown', 'Male_Graduate',
       'Male_High School', 'Male_Others', 'Male_University',
       'Male_Unknown'], dtype=object)

In [17]:
credit['DEFAULT'].value_counts()

0    23364
1     6636
Name: DEFAULT, dtype: int64

In [42]:
group[(group['SEX_ED']==l[0]) & (group['DEFAULT']==0)]['COUNT'][0]/23364 
group[(group['SEX_ED']==l[0]) & (group['DEFAULT']==1)]['COUNT'][1]

1130

In [43]:
np.log(group[(group['SEX_ED']==l[0]) & (group['DEFAULT']==0)]['COUNT'][0]/23364) - np.log(group[(group['SEX_ED']==l[0]) & (group['DEFAULT']==1)]['COUNT'][1]/6636)

0.24853196236667552

In [52]:
list(group[(group['SEX_ED']==l[i]) & (group['DEFAULT']==1)]['COUNT'])

[692]

In [69]:
l1 = []
woe = pd.DataFrame(columns = ['SEX_ED','woe','iv'])
count_0 = 0
count_1 = 0
for i in range(len(l)):
    count_0 = list(group[(group['SEX_ED']==l[i]) & (group['DEFAULT']==0)]['COUNT'])[0]
    count_1 = list(group[(group['SEX_ED']==l[i]) & (group['DEFAULT']==1)]['COUNT'])[0]
    woe1 = np.log(count_0/23364) - np.log(count_1/6636)
    iv = woe1*(count_0/23364 - count_1/6636)
    l1.append([l[i],woe1,iv])
    woe.append([l[i],woe1,iv])

In [70]:
print(l1)

[['Female_Graduate', 0.24853196236667552, 0.011940479047638964], ['Female_High School', -0.08627645241986537, 0.0007436768634600507], ['Female_Others', 1.9994095341716172, 0.005771075433733395], ['Female_University', -0.004883993659082764, 6.891869966437028e-06], ['Female_Unknown', 1.2720291819694296, 0.007876262129891209], ['Male_Graduate', 0.07782332189073937, 0.0008599041786180778], ['Male_High School', -0.28360819796657655, 0.005751716456035364], ['Male_Others', 0.9926047947566303, 0.0010160908914017728], ['Male_University', -0.22309923148811128, 0.009465582551748047], ['Male_Unknown', 1.2094125276217538, 0.004285641776875815]]


In [64]:
# for i in range(len(l1)):
#     woe.iloc[i] = l1[i]
woe

,SEX_ED,woe,iv


In [76]:
woe = pd.DataFrame(l1)

In [78]:
woe

,0,1,2
0,Female_Graduate,0.248532,0.011940
1,Female_High School,-0.086276,0.000744
2,Female_Others,1.999410,0.005771
3,Female_University,-0.004884,0.000007
4,Female_Unknown,1.272029,0.007876
5,Male_Graduate,0.077823,0.000860
6,Male_High School,-0.283608,0.005752
7,Male_Others,0.992605,0.001016
8,Male_University,-0.223099,0.009466
9,Male_Unknown,1.209413,0.004286


In [77]:
sum(woe[2])

0.04771732119936913

In [79]:
group = pd.DataFrame(credit.groupby('SEX_MAR')['DEFAULT'].value_counts()).rename(columns = {'DEFAULT':'COUNT'}).reset_index()
group

,SEX_MAR,DEFAULT,COUNT
0,Female_Married,0,6609
1,Female_Married,1,1860
2,Female_Others,0,148
3,Female_Others,1,44
4,Female_Single,0,7555
5,Female_Single,1,1856
6,Female_Unknown,0,37
7,Female_Unknown,1,3
8,Male_Married,0,3844
9,Male_Married,1,1346


- Female_Unknown and Male_Unknown are more prone to default

In [80]:
# WOE and IV
l = group['SEX_MAR'].unique()
l

array(['Female_Married', 'Female_Others', 'Female_Single',
       'Female_Unknown', 'Male_Married', 'Male_Others', 'Male_Single',
       'Male_Unknown'], dtype=object)

In [81]:
credit['DEFAULT'].value_counts()

0    23364
1     6636
Name: DEFAULT, dtype: int64

In [82]:
group[(group['SEX_MAR']==l[0]) & (group['DEFAULT']==0)]['COUNT'][0]/23364 
group[(group['SEX_MAR']==l[0]) & (group['DEFAULT']==1)]['COUNT'][1]

1860

In [83]:
np.log(group[(group['SEX_MAR']==l[0]) & (group['DEFAULT']==0)]['COUNT'][0]/23364) - np.log(group[(group['SEX_MAR']==l[0]) & (group['DEFAULT']==1)]['COUNT'][1]/6636)

0.009168864913341013

In [86]:
list(group[(group['SEX_MAR']==l[0]) & (group['DEFAULT']==1)]['COUNT'])

[1860]

In [87]:
l1 = []
woe = pd.DataFrame(columns = ['SEX_MAR','woe','iv'])
count_0 = 0
count_1 = 0
for i in range(len(l)):
    count_0 = list(group[(group['SEX_MAR']==l[i]) & (group['DEFAULT']==0)]['COUNT'])[0]
    count_1 = list(group[(group['SEX_MAR']==l[i]) & (group['DEFAULT']==1)]['COUNT'])[0]
    woe1 = np.log(count_0/23364) - np.log(count_1/6636)
    iv = woe1*(count_0/23364 - count_1/6636)
    l1.append([l[i],woe1,iv])

In [88]:
print(l1)

[['Female_Married', 0.009168864913341013, 2.3671742628906128e-05], ['Female_Others', -0.045664364004010416, 1.351521528703347e-05], ['Female_Single', 0.14509895751082924, 0.0063370762085782], ['Female_Unknown', 1.2536186201262502, 0.001418536378763404], ['Male_Married', -0.2093107439643549, 0.008017943132421324], ['Male_Others', -0.4367069514469506, 0.0009314289074135181], ['Male_Single', -0.031155513645635402, 0.00021386619286507268], ['Male_Unknown', 0.5330724653781909, 0.00011313093388810404]]


In [90]:
woe = pd.DataFrame(l1)

In [91]:
woe

,0,1,2
0,Female_Married,0.009169,0.000024
1,Female_Others,-0.045664,0.000014
2,Female_Single,0.145099,0.006337
3,Female_Unknown,1.253619,0.001419
4,Male_Married,-0.209311,0.008018
5,Male_Others,-0.436707,0.000931
6,Male_Single,-0.031156,0.000214
7,Male_Unknown,0.533072,0.000113


In [92]:
sum(woe[2])

0.01706916871184556